# Régler les hyperparamètres avec un travail de balayage

Beaucoup d’algorithmes de machine learning utilisent des hyperparamètres (valeurs de paramètres qui influencent l’entraînement, mais qui ne peuvent pas être déterminées à partir des données d’entraînement elles-mêmes). Par exemple, lors de l’entraînement d’un modèle de régression logistique, vous pouvez utiliser le taux de régularisation comme hyperparamètre pour compenser un biais du modèle. Ou bien, lors de l’entraînement d’un réseau neuronal convolutif, vous pouvez utiliser des hyperparamètres tels que le taux d’entraînement et la taille de lot pour contrôler le mode d’ajustement des pondérations et le nombre d’éléments de données traités dans un mini-lot, respectivement. Le choix des valeurs d’hyperparamètres peut affecter sensiblement les performances d’un modèle formé, ou le temps nécessaire pour effectuer l’apprentissage d’un modèle. Par ailleurs, souvent, vous devrez essayer plusieurs combinaisons pour trouver la solution optimale. 

## Avant de commencer

Vous devez avoir la dernière version du package **azureml-ai-ml** pour exécuter le code dans ce notebook. Exécutez la cellule ci-dessous pour vérifier qu’il est installé.

> **Remarque** :
> Si le package **azure-ai-ml** n’est pas installé, exécutez `pip install azure-ai-ml` pour l’installer.

In [ ]:
## Se connecter à un espace de travail

Une fois les packages requis du Kit de développement logiciel (SDK) installés, vous êtes prêt à vous connecter à votre espace de travail.

Pour vous connecter à un espace de travail, vous avez besoin de paramètres d’identificateur : un ID d’abonnement, un nom de groupe de ressources et un nom d’espace de travail. Le nom du groupe de ressources et le nom de l’espace de travail sont déjà renseignés pour vous. Vous avez seulement besoin de l’ID d’abonnement pour exécuter la commande.

Pour trouver les paramètres nécessaires, cliquez sur l’abonnement et le nom de l’espace de travail en haut à droite du studio. Un volet s’ouvre à droite.

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> Copiez l’ID d’abonnement et remplacez **YOUR-SUBSCRIPTION-ID** par la valeur que vous avez copiée. </p>

## Créer le script d’apprentissage
Le réglage des hyperparamètres est idéal lorsque vous souhaitez entraîner des modèles Machine Learning, mais en faisant varier les paramètres d’entrée. Vous devez créer un script d’entraînement qui attend un paramètre d’entrée représentant l’un des hyperparamètres de l’algorithme.

Exécutez les cellules suivantes pour créer le dossier **src** et le script d’entraînement.

Notez que le script d’entraînement attend deux paramètres d’entrée :

- `--training_data`, qui attend une chaîne. Vous spécifiez le chemin vers une ressource de données inscrite en tant que données d’entraînement d’entrée.
- `--reg_rate`, qui attend un nombre, mais a une valeur par défaut de `0.01`. Vous utilisez ce paramètre d’entrée pour le réglage des hyperparamètres.

In [ ]:
## Configurer et exécuter un travail de commande

Exécutez la cellule ci-dessous pour entraîner un modèle de classification afin de prédire le diabète. Le modèle est entraîné par l’exécution du script **train\.py** qui se trouve dans le dossier **src**. Il utilise la ressource de données `diabetes-data` inscrite comme données d’entraînement. 

- `code` : spécifie le dossier contenant le script à exécuter.
- `command` : spécifie ce qu’il faut exécuter exactement.
- `environment` : spécifie les packages nécessaires qui doivent être installés sur la capacité de calcul avant d’exécuter la commande.
- `compute` : spécifie la capacité de calcul à utiliser pour exécuter la commande.
- `display_name` : nom du travail individuel.
- `experiment_name` : nom de l’expérience à laquelle le travail appartient.

Notez que le travail de commande n’exécute le script d’entraînement qu’une seule fois, avec un taux de régularisation de `0.1`. Avant d’exécuter un travail de balayage pour régler les hyperparamètres, il est recommandé de tester votre script pour vérifier qu’il fonctionne comme prévu avec un travail de commande.

In [ ]:
## Définir l’espace de recherche

Une fois votre travail de commande terminé, vous pouvez configurer et exécuter un travail de balayage. 

Tout d’abord, vous devez spécifier l’espace de recherche de votre hyperparamètre. Pour entraîner trois modèles, chacun avec un taux de régularisation différent (`0.01`, `0.1` ou `1`), définissez l’espace de recherche avec un hyperparamètre `Choice`. 

## Configurer et envoyer le travail de balayage

Vous utilisez la fonction de balayage pour effectuer le réglage des hyperparamètres sur votre script d’entraînement. Pour configurer un travail de balayage, vous devez configurer les éléments suivants :

- `compute` : nom de la capacité de calcul cible sur laquelle exécuter le travail.
- `sampling_algorithm` : algorithme d’échantillonnage des hyperparamètres à utiliser sur l’espace de recherche. Les valeurs autorisées sont `random`, `grid` et `bayesian`.
- `primary_metric` : nom de la métrique principale journalisée par chaque travail d’essai. La métrique doit être enregistrée dans le script de formation de l’utilisateur en utilisant `mlflow.log_metric()` avec le nom de métrique correspondant.
- `goal` : objectif d’optimisation de `primary_metric`. Les valeurs autorisées sont `maximize` et `minimize`.
- `limits` : limites du travail de balayage. Par exemple, la quantité maximale d’essais ou de modèles que vous souhaitez entraîner.

Notez que le travail de commande est utilisé comme base pour le travail de balayage. La configuration du travail de commande sera réutilisée par le travail de balayage.

In [ ]:
Exécutez la cellule suivante pour soumettre le travail de balayage.

In [ ]:
Une fois le travail terminé, accédez à la vue d’ensemble du travail. L’onglet **Essais** affiche tous les modèles qui ont été entraînés ainsi que les écarts de score `Accuracy` pour chaque valeur de taux de régularisation que vous avez essayée.

## Configure and run a command job

Run the cell below to train a classification model to predict diabetes. The model is trained by running the **train\.py** script that can be found in the **src** folder. It uses the registered `diabetes-data` data asset as the training data. 

- `code`: specifies the folder that includes the script to run.
- `command`: specifies what to run exactly.
- `environment`: specifies the necessary packages to be installed on the compute before running the command.
- `compute`: specifies the compute to use to run the command.
- `display_name`: the name of the individual job.
- `experiment_name`: the name of the experiment the job belongs to.

Note that the command job only runs the training script once, with a regularization rate of `0.1`. Before you run a sweep job to tune hyperparameters, it's a best practice to test whether your script works as expected with a command job.

In [ ]:
from azure.ai.ml import command, Input
from azure.ai.ml.constants import AssetTypes

# configure job

job = command(
    code="./src",
    command="python train.py --training_data ${{inputs.diabetes_data}} --reg_rate ${{inputs.reg_rate}}",
    inputs={
        "diabetes_data": Input(
            type=AssetTypes.URI_FILE, 
            path="azureml:diabetes-data:1"
            ),
        "reg_rate": 0.01,
    },
    environment="AzureML-sklearn-0.24-ubuntu18.04-py37-cpu@latest",
    compute="aml-cluster",
    display_name="diabetes-train-mlflow",
    experiment_name="diabetes-training", 
    tags={"model_type": "LogisticRegression"}
    )

# submit job
returned_job = ml_client.create_or_update(job)
aml_url = returned_job.studio_url
print("Monitor your job at", aml_url)

## Define the search space

When your command job has completed successfully, you can configure and run a sweep job. 

First, you'll need to specify the search space for your hyperparameter. To train three models, each with a different regularization rate (`0.01`, `0.1`, or `1`), you can define the search space with a `Choice` hyperparameter. 

In [ ]:
from azure.ai.ml.sweep import Choice

command_job_for_sweep = job(
    reg_rate=Choice(values=[0.01, 0.1, 1]),
)

## Configure and submit the sweep job

You'll use the sweep function to do hyperparameter tuning on your training script. To configure a sweep job, you'll need to configure the following:

- `compute`: Name of the compute target to execute the job on.
- `sampling_algorithm`: The hyperparameter sampling algorithm to use over the search space. Allowed values are `random`, `grid` and `bayesian`.
- `primary_metric`: The name of the primary metric reported by each trial job. The metric must be logged in the user's training script using `mlflow.log_metric()` with the same corresponding metric name.
- `goal`: The optimization goal of the `primary_metric`. The allowed values are `maximize` and `minimize`.
- `limits`: Limits for the sweep job. For example, the maximum amount of trials or models you want to train.

Note that the command job is used as the base for the sweep job. The configuration for the command job will be reused by the sweep job.

In [ ]:
# apply the sweep parameter to obtain the sweep_job
sweep_job = command_job_for_sweep.sweep(
    compute="aml-cluster",
    sampling_algorithm="grid",
    primary_metric="training_accuracy_score",
    goal="Maximize",
)

# set the name of the sweep job experiment
sweep_job.experiment_name="sweep-diabetes"

# define the limits for this sweep
sweep_job.set_limits(max_total_trials=4, max_concurrent_trials=2, timeout=7200)

Run the following cell to submit the sweep job.

In [ ]:
returned_sweep_job = ml_client.create_or_update(sweep_job)
aml_url = returned_sweep_job.studio_url
print("Monitor your job at", aml_url)

When the job is completed, navigate to the job overview. The **Trials** tab will show all models that have been trained and how the `Accuracy` score differs for each regularization rate value you tried.